In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import configparser

In [48]:
#read config file
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [49]:
driver_path = config['webdriver']['path']
base_url = config['webdriver']['base_url']

In [50]:
driver = webdriver.Firefox(executable_path = driver_path)
driver.set_page_load_timeout(15)

In [71]:
def get_contents_list():
    content = driver.find_element_by_id('content')
    program_wrap = content.find_element_by_class_name('program_wrap')
    program_list = program_wrap.find_element_by_id('cds_flick')
    container = program_list.find_element_by_class_name('flick-container')
    container_area = container.find_element_by_class_name('program_all')
    
    daily_program_list = container_area.find_elements_by_class_name('col')
    
    for col in daily_program_list:
        anchors = col.find_elements_by_css_selector('a')
        for anchor in anchors:
            href = anchor.get_attribute('href')
            print(href)

In [72]:
driver.get(base_url)
driver.implicitly_wait(10)
get_contents_list()

http://tv.naver.com/jtbc.hyolee/playlists
http://tv.naver.com/jtbc.hyolee/playlists
http://tv.naver.com/superman/playlists
http://tv.naver.com/superman/playlists
http://tv.naver.com/kbs.dad/playlists
http://tv.naver.com/kbs.dad/playlists
http://tv.naver.com/sbs.woori/playlists
http://tv.naver.com/sbs.woori/playlists
http://tv.naver.com/inkigayo/playlists
http://tv.naver.com/inkigayo/playlists
http://tv.naver.com/mask/playlists
http://tv.naver.com/mask/playlists
http://tv.naver.com/kbs.ssammyway/playlists
http://tv.naver.com/kbs.ssammyway/playlists
http://tv.naver.com/cjenm.habaek2017/playlists
http://tv.naver.com/cjenm.habaek2017/playlists
http://tv.naver.com/sbs.mysassygirl/playlists
http://tv.naver.com/sbs.mysassygirl/playlists
http://tv.naver.com/kbs.namegirl/playlists
http://tv.naver.com/kbs.namegirl/playlists
http://tv.naver.com/idoldrama/playlists
http://tv.naver.com/idoldrama/playlists
http://tv.naver.com/nonsummit/playlists
http://tv.naver.com/nonsummit/playlists
http://tv.nave